In [1]:
from helpers import load_all_datasets, compose, str_rep_vdb, str_rep_no_steps, CBRDatabase, str_rep_vdb_ing, str_rep_vdb_nam 
import pandas as pd
additional_cbr_1, additional_cbr_2, additional_validation, cbr_augmentation, cbr_database, model_training, validation = load_all_datasets()

DATABASE_NAME = "cbr_database_10_100"
OUTPUT_FILE = "cbr_10_100_gpt2_input"

IX_FILE1 = "cbr_augmentation_1_gpt2_results_p_ix.csv"
IX_FILE2 = "cbr_augmentation_2_gpt2_results_p_ix.csv"
dfix1 = pd.read_csv(f"{IX_FILE1}")
dfix2 = pd.read_csv(f"{IX_FILE2}")
cbr_id = cbr_database.set_index("id")

In [2]:
cbr_db = pd.concat([additional_cbr_1, additional_cbr_2])
cbr_db_ix = pd.concat([dfix1, dfix2])
cbr_db_ix = cbr_db_ix.set_index("id")

large_cbr_reps = str_rep_vdb(cbr_db)
len(large_cbr_reps)

10626

In [3]:
cbr_db_ix

,steps,ingredients,name
id,,,
277643,Cover chicken with water in a large pot. Cook ...,"cabbage, russet potatoes, cooked ham, chicken ...",potato cabbage soup
54179,Preheat oven to 350f. Prepare the caramel appl...,"apples, baking mix, cinnamon, caramels, nuts, ...",caramel apple bake
168492,Combine all the ingredients except the lobster...,"lemon juice, olive oil, salt, paprika, black p...",grilled rock lobster tails
95061,Combine sugar and water and bring to the boil....,"water, sugar, frozen raspberries, mango, fresh...",valentine granita
44739,Preheat oven to 350f. In a medium bowl mix sug...,"sugar, water, orange juice concentrate, pecan ...",orange sugared pecans
...,...,...,...
122286,Preheat oven to 350f line a 12-cup muffin tin ...,"all-purpose flour, baking powder, salt, butter...",south carolina strawberry shortcake
53152,"In dutch oven or electric skillet, heat oil. A...","nacho cheese soup, cheddar cheese, milk, prepa...",souper easy macaroni and cheese
458446,"In a medium saucepan, combine 2 cups water, su...","water, sugar, lemongrass, fresh ginger, lemon,...",house of blues ginger lemongrass soda water


In [4]:
# large_cbr = pd.concat([cbr_database, cbr_augmentation, additional_cbr_1, additional_cbr_2])
# large_cbr_id = large_cbr.set_index("id")
# large_cbr_reps = str_rep_vdb(large_cbr_id)
# len(large_cbr_reps)

In [5]:
meta = CBRDatabase.create_metadatas([int(e) for e in list(cbr_db["id"].values)], {"dataset":"cbr_10_100"})
vdb = CBRDatabase(large_cbr_reps, metadatas=meta, db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=True)

# vdb = CBRDatabase(db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=False)
print(len(vdb.get_texts()))
print(vdb.similarity_search(query="test", k=1))

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
10626
[Document(page_content='name: sweet potato surprise balls oamc ; ingredients: sweet potatoes, eggs, butter, orange rind, brown sugar, vanilla, marshmallows, corn flake crumbs ; preparation:', metadata={'dataset': 'cbr_10_100', 'recipe_id': 109548})]


In [6]:
# generowanie inputów - 850 tokenów 
import random
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

inputs = []
pe_recipes = []
target_recipe = []
lens = []

for i, v_recipe in validation.iterrows():
    while True:
        tem = vdb.similarity_search(query=str_rep_no_steps(v_recipe), k=5)
        recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in tem]
        # print(tem)#dev
        # print(compose(cbr_db_ix.loc[[recipe_ixs[0]]], v_recipe))#dev
        # raise

        gen_input = compose(cbr_db_ix.loc[recipe_ixs], v_recipe)
        br = False
        if len(tokenizer.encode(gen_input)) <= 850: 
            br = True
        else:
            for ii in list(range(4, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(3, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(2, -1, -1)):
                gen_input = compose(cbr_db_ix.loc[recipe_ixs[:ii] + recipe_ixs[ii+3:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 850: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
        
        if not br:
            print(cbr_db_ix.loc[recipe_ixs].steps.values)
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        else:
            lens.append(len(tokenizer.encode(gen_input)))
            break
            
        
    pe_recipes.append(recipe_ixs)
    target_recipe.append(v_recipe["id"])
    inputs.append(gen_input)

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1 - 733
1 - 619
1 - 686
1 - 582
1 - 796
1 - 624
1 - 653
2 - 630
1 - 664
1 - 694
1 - 712
1 - 468
1 - 761
1 - 812
1 - 659
1 - 675
1 - 695
1 - 772
1 - 761
1 - 811
1 - 787
1 - 819
1 - 694
1 - 738
1 - 725
1 - 730
1 - 695
1 - 705
1 - 702
2 - 627
1 - 719
1 - 758
1 - 807
1 - 773
2 - 825
1 - 624
1 - 705
1 - 639
1 - 746
1 - 736
1 - 734
1 - 684
1 - 665
1 - 727
1 - 718
1 - 736
1 - 690
1 - 766
1 - 778
1 - 745
1 - 676
1 - 679
2 - 657
1 - 799
1 - 776
1 - 736
1 - 796
1 - 805
1 - 707
1 - 722
1 - 804
1 - 609
1 - 702
1 - 705
1 - 780
1 - 783
1 - 700
1 - 722
1 - 797
2 - 654
1 - 767
1 - 717
1 - 788
2 - 525
1 - 727
1 - 729
1 - 706
1 - 750
1 - 822
1 - 696
1 - 798
1 - 765
1 - 690
1 - 805
1 - 750
1 - 720
1 - 736
1 - 717
1 - 698
1 - 678
1 - 636
1 - 684
1 - 730
1 - 760
2 - 698
1 - 818
1 - 741
1 - 741
1 - 754
1 - 678
1 - 810
1 - 725
1 - 719
1 - 816
1 - 748
1 - 699
1 - 774
1 - 716
1 - 709
1 - 604
1 - 731
1 - 843
1 - 605
1 - 742
1 - 616
1 - 762
1 - 770
1 - 683
1 - 527
1 - 700
1 - 650
1 - 735
1 - 797
1 - 736
1 - 741


Token indices sequence length is longer than the specified maximum sequence length for this model (2496 > 2048). Running this sequence through the model will result in indexing errors


1 - 807
1 - 826
1 - 788
1 - 842
1 - 762
2 - 282
1 - 788
1 - 793
1 - 742
1 - 760
1 - 728
1 - 789
1 - 696
1 - 754
1 - 720
1 - 748
1 - 746
1 - 621
1 - 824
1 - 732
1 - 669
1 - 742
1 - 711
1 - 568
1 - 742
1 - 728
1 - 754
1 - 590
1 - 681
1 - 743
1 - 805
1 - 705
1 - 758
1 - 763
1 - 567
1 - 691
1 - 687
1 - 822
1 - 647
1 - 803
1 - 717
1 - 788
2 - 803
1 - 732
1 - 591
1 - 724
1 - 794
1 - 802
1 - 654
1 - 822
1 - 782
1 - 664
1 - 695
1 - 830
1 - 788
1 - 640
1 - 733
1 - 799
1 - 645
1 - 770
1 - 725
1 - 741
1 - 741
1 - 811
1 - 707
1 - 730
1 - 826
1 - 593
1 - 802
1 - 643
1 - 710
1 - 749
1 - 791
1 - 672
1 - 684
1 - 714
1 - 781
1 - 844
1 - 793
1 - 707
1 - 701
1 - 763
1 - 611
1 - 745
1 - 772
1 - 698
1 - 654
1 - 842
1 - 798
1 - 849
1 - 805
1 - 701
1 - 669
1 - 705
1 - 823
1 - 768
1 - 719
1 - 717
1 - 844
1 - 705
1 - 709
1 - 661
1 - 830
1 - 725
1 - 764
1 - 732
1 - 690
1 - 784
1 - 702
1 - 774
1 - 737
1 - 709
1 - 800
1 - 676
1 - 728
1 - 787
1 - 593
1 - 789
1 - 727
1 - 753
1 - 761
1 - 848
1 - 823
1 - 831
1 - 786


In [7]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{OUTPUT_FILE}.csv", index=False)

In [8]:
vdb.similarity_search(query="str_rep_no_steps(v_recipe)", k=7)

[Document(page_content='name: vegan mushroom stroganoff ; ingredients: boca meatless ground burger, vegan sour cream, vegetable broth, mixed mushrooms, campanelle pasta, water, garlic powder, cumin, paprika, salt & freshly ground black pepper ; preparation:', metadata={'dataset': 'cbr_10_100', 'recipe_id': 225684}),
 Document(page_content='name: naked tater all the way ; ingredients: potatoes, butter, sour cream, milk, bacon bits, cheddar cheese, green onions, garlic powder, salt and pepper ; preparation:', metadata={'dataset': 'cbr_10_100', 'recipe_id': 135691}),
 Document(page_content='name: lentil waffles ; ingredients: green gram, rava, spinach, green chilies, fresh ginger, garlic, baking soda, oil, salt ; preparation:', metadata={'dataset': 'cbr_10_100', 'recipe_id': 145498}),
 Document(page_content='name: sweet and sour beef bake ; ingredients: boneless beef chuck roast, salt and pepper, oil, butter, onions, fresh garlic, sugar, red wine vinegar, prune ; preparation:', metadata={

In [9]:
from collections import Counter


Counter(df.input.apply(lambda x: x.split("name:")).apply(lambda x: len(x)))

Counter({6: 1047, 7: 1026, 5: 42, 4: 10})